# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [2]:

import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics
%pylab inline
ROBOT_POSE_DATA_DIR = 'robot_pose_data'

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [5]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Left', 'StandInit', 'Frog', 'Crouch', 'HeadBack', 'Back', 'Sit', 'Knee', 'Stand', 'Right', 'Belly']


In [16]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename, 'rb'))
    target = [i] * len(data)
    return data, target

In [24]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i, name in enumerate(classes):
    d,t=load_pose_data(i)
    all_data.extend(d)
    all_target.extend(t)
load_pose_data(i)
print(all_data[50])
print(all_target[50])
print ('total number of data', len(all_data))

[-0.022968053817749023, -0.024502038955688477, -0.23772811889648438, 0.8958139419555664, -0.022968053817749023, -0.02143406867980957, -0.5152546167373657, 0.941917896270752, -0.0339391864836216, 0.07631523162126541]
1
total number of data 222


In [25]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

array([  3,   4,   5,   9,  12,  19,  23,  28,  29,  37,  38,  40,  45,
        47,  49,  50,  51,  52,  53,  54,  55,  69,  70,  71,  72,  74,
        75,  77,  79,  85,  88,  89,  91,  93,  94, 105, 111, 115, 118,
       122, 126, 127, 128, 133, 134, 135, 143, 147, 148, 150, 151, 156,
       158, 167, 169, 177, 178, 188, 191, 195, 198, 202, 206, 212, 214,
       215, 221])

In [32]:
x_train=np.asarray(all_data)[permutation]
y_train=np.asarray(all_target)[permutation]


10
0


In [27]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [34]:
# YOUR CODE HERE
print(clf.fit(x_train,y_train))

SVC(C=100.0, gamma=0.001)


### predicting

In [37]:
clf.predict([all_data[-1]])[0], [all_target[-1]][0]

(10, 10)

In [38]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [44]:
expected = []
predicted = []
# YOUR CODE HERE

predicted=clf.predict(x_train)
expected=y_train
evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        52
           2       1.00      1.00      1.00        10
           3       0.97      1.00      0.98        30
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00        23
           6       1.00      1.00      1.00        26
           7       1.00      1.00      1.00        10
           8       1.00      1.00      1.00        11
           9       1.00      0.91      0.95        11
          10       1.00      1.00      1.00        19

    accuracy                           1.00       222
   macro avg       1.00      0.99      0.99       222
weighted avg       1.00      1.00      1.00       222


Confusion matrix:
[[20  0  0  0  0  0  0  0  0  0  0]
 [ 0 52  0  0  0  0  0  0  0  0  0]
 [ 0  0 10  0  0  0  0  0  0  0  0]
 [ 0  0  0 30  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [45]:
expected = []
predicted = []
test_index=np.delete(np.arange(len(all_data)),training_data)
x_test=np.asarray(all_data)[test_index]
y_test=np.asarray(all_target)[test_index]

predicted=clf.predict(x_test)
expected=y_test

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        18
           2       1.00      1.00      1.00         5
           3       0.89      1.00      0.94         8
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         8
           6       1.00      1.00      1.00         8
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       1.00      0.67      0.80         3
          10       1.00      1.00      1.00         5

    accuracy                           0.99        67
   macro avg       0.99      0.97      0.98        67
weighted avg       0.99      0.99      0.98        67


Confusion matrix:
[[ 6  0  0  0  0  0  0  0  0  0  0]
 [ 0 18  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0]
 [ 0  0  0  8  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [47]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [62]:
clf2 = pickle.load(open(ROBOT_POSE_CLF,'rb'))
clf2.predict([x_test[-1]])[0], y_test[-1]

(0, 0)